# MATH50003 Numerical Analysis (2022–23)
# Lab 3: Divided differences and dual numbers

This lab explores different discretisations for first and higher derivatives.
In particular we consider the following approximations:
*Forward differences*:
$$
f'(x) ≈ {f(x+h) - f(x) \over h}
$$
*Central differences*:
$$
f'(x) ≈ {f(x+h) - f(x-h) \over 2h}
$$
*Second order differences*:
$$
f''(x) ≈ {f(x+h) - 2f(x) + f(x-h) \over h^2}
$$
We also add to the implementation of `Dual` to enable
automatic differentiation with cos, sin, and division

In [1]:
using Plots, Test
nanabs = x -> iszero(x) ? NaN : abs(x)

f = x -> 1 + x + x^2
fp = x -> 1 + 2x
g = x -> 1 + x/3 + x^2
gp = x -> 1/3 + 2*x

h1 = 2.0 .^ (0:-1:-60)
h2 = 10.0 .^ (0:-1:-16)

#PROBLEM 1
central_f_h1(x) = ((f.(x .+ h1) .- f.(x .- h1))./(2 .* h1)) .- fp(x)
central_g_h1(x) = ((g.(x .+ h1) .- g.(x .- h1))./(2 .* h1)) .- gp(x)

x = 0.0
#plot(nanabs.(central_f_h1(x)), yaxis=:log10, label="f")
#plot!(nanabs.(central_g_h1(x)), yaxis=:log10, label="g")


# MAKE FUNCTION:
central_diff(x, h, f) = (f(x + h) - f(x - h)) / (2* h)
central_diff_err(x, h, f, fp) = nanabs(central_diff(x, h, f) - fp(x))
    
#plot(central_diff_err.(0.0, h1, f, fp), yaxis=:log10, label="f")
#plot!(central_diff_err.(0.0, h1, g, gp), yaxis=:log10, label="g")
       

central_diff_err (generic function with 1 method)

--------

**Problem 1** Implement central differences
for $f(x) = 1 + x + x^2$ and $g(x) = 1 + x/3 + x^2$, approximating the derivative at $x = 0$.
Plot the absolute errors for `h = 2.0 .^ (0:-1:-60)` and `h = 10.0 .^ (0:-1:-16)`.

Hint: the easiest way to do this is the copy the code from lectures/notes for forward differences,
and replace the line `f.(h) .- f(0) ./ h` with the equivalent for central differences.
Note that `f.(h)` is broadcast notation so creates a vector containing `[f(h[1]),…,f(h[end])]`.
Thus that expression creates a vector containing `[(f(h[1])-f(0))/h[1], …, (f(h[end])-f(0))/h[end]]`.

-----
**Problem 2** Use forward differences, central differences, and second-order divided differences to approximate to 5-digits the first and second
derivatives to the following functions
at the point $x = 0.1$:
$$
\exp(\exp x \cos x + \sin x), ∏_{k=1}^{1000} \left({x \over k}-1\right), \hbox{ and } f^{\rm s}_{1000}(x)
$$
where $f^{\rm s}_n(x)$ corresponds to $n$-terms of the following continued fraction:
$$
1 + {x-1 \over 2 + {x-1 \over 2 + {x-1 \over 2 + ⋱}}},
$$
e.g.:
$$f^{\rm s}_1(x) = 1 + {x-1 \over 2}$$
$$f^{\rm s}_2(x) = 1 + {x-1 \over 2 + {x -1 \over 2}}$$
$$f^{\rm s}_3(x) = 1 + {x-1 \over 2 + {x -1 \over 2 + {x-1 \over 2}}}$$

----

We now extend our implementation of `Dual` which we began in lectures/notes as follows:

In [2]:
#PROBLEM 2:
h = x -> exp((exp(x)*cos(x)) + sin(x))
    
ip = x -> prod([x] ./ (1:1000) .- 1)
# so this takes x and makes it a vector of 1000 columns then it  
# divides it by each of 1:1000 and subtracts 1
# then finds the product
"""
function i(x)
    ret = 1 
    for k=1:1000
        ret = ret * ((x / k) - 1)
    end
    ret
end """
   
function j(x, k)
    if k == 0
        ret = 0
    else 
        ret = 1 + (x - 1)/(2 + j(x, k-1))
    end
    ret 
end 


forward_diff(x, h, f) = (f(x + h) - f(x)) / h
central_diff(x, h, f) = (f(x + h) - f(x - h)) / (2* h)
second_ord_diff(x, h, f) = (f(x + h) - 2*f(x) + f(x - h)) / (h ^ 2)

x = 0.1
e = eps()
println(forward_diff(x, sqrt(e), h))
println(central_diff(x, sqrt(e), h))
println(second_ord_diff(x, e, h))
        
println(forward_diff(x, sqrt(e), ip))
println(central_diff(x, sqrt(e), ip))
println(second_ord_diff(x, e, ip))
        
println(forward_diff(x, sqrt(e), x -> j(x, 1000)))
println(central_diff(x, sqrt(e), x -> j(x, 1000)))
println(second_ord_diff(x, e, x -> j(x, 1000)))
        

# Dual(a,b) represents a + b*ϵ
struct Dual{T}
    a::T
    b::T
end

# Dual(a) represents a + 0*ϵ
Dual(a::Real) = Dual(a, zero(a)) # for real numbers we use a + 0ϵ

# Allow for a + b*ϵ syntax
const ϵ = Dual(0, 1)

# import the functions which we wish to overload
import Base: +, *, -, /, ^, zero, exp, cos, sin, one

# support polynomials like 1 + x, x - 1, 2x or x*2 by reducing to Dual
+(x::Real, y::Dual) = Dual(x) + y
+(x::Dual, y::Real) = x + Dual(y)
-(x::Real, y::Dual) = Dual(x) - y
-(x::Dual, y::Real) = x - Dual(y)
*(x::Real, y::Dual) = Dual(x) * y
*(x::Dual, y::Real) = x * Dual(y)

# support x/2 (but not yet division of duals)
/(x::Dual, k::Real) = Dual(x.a/k, x.b/k)

# a simple recursive function to support x^2, x^3, etc.
function ^(x::Dual, k::Integer)
    if k < 0
        error("Not implemented")
    elseif k == 1
        x
    else
        x^(k-1) * x
    end
end

# support identity of type Dual
one(x::Dual) = Dual(one(eltype(x.a)))

# Algebraic operations for duals
-(x::Dual) = Dual(-x.a, -x.b)
+(x::Dual, y::Dual) = Dual(x.a + y.a, x.b + y.b)
-(x::Dual, y::Dual) = Dual(x.a - y.a, x.b - y.b)
*(x::Dual, y::Dual) = Dual(x.a*y.a, x.a*y.b + x.b*y.a)

exp(x::Dual) = Dual(exp(x.a), exp(x.a) * x.b)

6.584772557020187
6.584772542119026
-9.007199254740992e15
-3.593826290220022
-3.5938265174627304
3.377699720527872e15
0.4303314760327339
0.4303314797580242
-4.503599627370496e15


exp (generic function with 16 methods)

**Problem 3.1** Add support for `cos`, `sin`, and `/` to the type `Dual`

In [10]:
function cos(x::Dual)
    Dual(cos(x.a), - x.b*sin(x.a))
end

function sin(x::Dual)
    Dual(sin(x.a), x.b*cos(x.a))
end

function /(x::Dual, y::Dual)
    Dual(x.a/y.a, ((x.b * y.a) - (x.a * y.b))/(y.a^2))
end

x = 0.1
@test cos(sin(x+ϵ)/(x+ϵ)).b ≈ -((cos(x)/x - sin(x)/x^2)sin(sin(x)/x))

fdual = f(Dual(0.1, 1))
print(fdual.b)

LoadError: MethodError: no method matching Dual(::Float64, ::Int64)
[0mClosest candidates are:
[0m  Dual(::Real) at In[2]:53
[0m  Dual(::T, [91m::T[39m) where T at In[2]:48

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*